### Loading the model:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "openlm-research/open_llama_7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
global_hook_handle = None
print("OpenLLaMA-7b loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

OpenLLaMA-7b loaded successfully!


### Loading the dataset and connect to google drive

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from google.colab import drive
import torch
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/MSBD5002_project/' #modify this line for your drive

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load the TriviaQA dataset
trivia_qa = load_dataset('trivia_qa', 'rc.nocontext')


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

train-00000-of-00001.parquet:   0%|          | 0.00/55.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/7.34M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm
full_dataset = []
for obs in tqdm(trivia_qa['train']):
  aliases = []
  aliases.extend(obs['answer']['aliases'])
  aliases.extend(obs['answer']['normalized_aliases'])
  aliases.append(obs['answer']['value'])
  aliases.append(obs['answer']['normalized_value'])
  full_dataset.append((obs['question'], aliases))

100%|██████████| 138384/138384 [00:24<00:00, 5761.64it/s]


In [ ]:
data = full_dataset[0:2] # choose the size of the datasets

### Get the activations score (implementation)
##### if you already have the data and want to process the classifer, you can directly jump to the section of classifier

In [ ]:
import torch
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

model.to('cuda')
model.eval()

# use a hook to extract the activaition score
def hook(module, input, output):
    activations.append(output.detach())
activations_total_list = []


if global_hook_handle is not None:
  global_hook_handle.remove()

global_hook_handle = model.model.layers[-1].mlp.register_forward_hook(hook)

# Looping the data
for item in tqdm(data):
  prompt = item[0]
  answer = item[1]

  #initialization
  qm_a1_scores = []
  response = []
  activations = []
  inputs_id = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
  start_pos = inputs_id.shape[-1]
  num_layers = len(model.model.layers)
  idx = num_layers - 1
  prompt_len = len(inputs_id[0])


  #getting the predicted token
  max_length = 2
  bar = range(max_length)

  for i in bar:
    with torch.no_grad():
        outputs = model(inputs_id, output_hidden_states=True)
    next_token = outputs.logits[:, -1, :].argmax(dim=-1)
    inputs_id = torch.cat([inputs_id, next_token.unsqueeze(-1)], dim=-1)
    response.append(next_token)



  # Store the activations weights
  last_layer_activations = activations
  activations_total_list.append(last_layer_activations)
  # shape of activations_total_list:
  # (num_prompt, max_length, batch_size, prompt_len, 3200)


activations_scores = [x[1][0][-1] for x in activations_total_list] # extrcat the required score

save_data = {
    "activations_scores": activations_scores,
}
torch.save(save_data, data_path + f"activations_data 9000.pt") # store the result to google drive




AttributeError: 'Sequential' object has no attribute 'to'

### Get the activations score (Research part - stacked approach)
This part is independent to the implementation part

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy

# Load BERT-based NER model
tokenizer_bert = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model_bert = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_pipeline = pipeline("ner", model=model_bert, tokenizer=tokenizer_bert, aggregation_strategy="simple")

# Load spaCy for fallback POS and dependency parsing
nlp = spacy.load("en_core_web_sm")

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
"""
Here is the part for Extractive summarization
"""
def extract_keyword(text):
    # Step 1: Run BERT-based NER
    ner_results = ner_pipeline(text)

    # Extract the first entity (PERSON, GPE, ORG, etc.)
    for entity in ner_results:
        return entity["word"]  # e.g., "Sinclair", "England", "Super Bowl XX"

    # Step 2: Fallback to spaCy for non-entity nouns (e.g., "country")
    doc = nlp(text)
    for token in doc:
        # Prioritize proper nouns (PROPN) or significant nouns (NOUN)
        if token.pos_ in ["PROPN", "NOUN"] and token.dep_ in ["nsubj", "dobj", "pobj", "ROOT"]:
            # Handle multi-word proper nouns (e.g., "David Soul")
            if token.pos_ == "PROPN" and token.head.pos_ == "PROPN" and token.head != token:
                return f"{token.text} {token.head.text}"
            return token.text  # Single noun (e.g., "country", "William")

    return "No keyword found"

# Demo for keyword extraction
prompts= [data[i][0] for i in range(1)]
for prompt in prompts:
    keyword = extract_keyword(prompt)
    print(f"Prompt: {prompt} -> Keyword: {keyword}")

Prompt: Which American-born Sinclair won the Nobel Prize for Literature in 1930? -> Keyword: American
Prompt: Where in England was Dame Judi Dench born? -> Keyword: England


In [ ]:
import torch
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

model.to('cuda')
model.eval()

# use a hook to extract the activation score
def hook(module, input, output):
    activations.append(output.detach())


if global_hook_handle is not None:
  global_hook_handle.remove()

global_hook_handle = model.model.layers[-1].mlp.register_forward_hook(hook)

#initialization
activations_total_list = []
activations_scores_qm = []
activations_scores_a1 = []
activations_scores_a1vsqm = []

for item in tqdm(data):
  prompt = item[0]
  answer = item[1]
  stop_token = 13

  #initialization
  response = []
  attention_scores = []
  activations = []
  inputs_id = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
  start_pos = inputs_id.shape[-1]
  num_layers = len(model.model.layers)
  idx = num_layers - 1
  prompt_len = len(inputs_id[0])
  keyword_id = tokenizer(extract_keyword(prompt), return_tensors="pt").input_ids.to("cuda")[0][1].item()
  original_inputs_id = inputs_id.clone()

  qm_idx = prompt_len - 1 # finding position of the keyword
  for k in range(prompt_len):
    if keyword_id == inputs_id[0][k].item():
      qm_idx = k



  #getting the predicted token
  max_length = prompt_len * 3
  bar = range(max_length)
  stop_token = 13

  for i in bar:
    with torch.no_grad():
        outputs = model(inputs_id, output_hidden_states=True, output_attentions=True)
    next_token = outputs.logits[:, -1, :].argmax(dim=-1)
    inputs_id = torch.cat([inputs_id, next_token.unsqueeze(-1)], dim=-1)
    response.append(next_token)

    # Store the attention weights
    last_layer_attn = outputs.attentions[-1].detach().cpu()
    attention_scores.append(last_layer_attn)

    # break the loop if the stop token is reached
    if next_token.item() == stop_token and i > 5:
      response_list = torch.stack(response).cpu().numpy()
      str_response = tokenizer.decode(response_list.flatten(), skip_special_tokens=True)
      break

    if i == max_length - 1:
      response_list = torch.stack(response).cpu().numpy()
      str_response = tokenizer.decode(response_list.flatten(), skip_special_tokens=True)

  # Remove the repeated word that appearing in the input prompt for the generated response
  words_to_remove = prompt.split()
  filtered_output = str_response
  for word in words_to_remove:
    filtered_output = filtered_output.replace(word, "")

  # Find the index for the keyword in the response
  a1_idx = prompt_len
  if extract_keyword(filtered_output) != "No keyword found":
    keyword_id = tokenizer(extract_keyword(filtered_output), return_tensors="pt").input_ids.to("cuda")[0][1].item()
    for k in range(len(inputs_id[0]) - 1 - prompt_len):
      if keyword_id == inputs_id[0][k + prompt_len].item():
        a1_idx = k + prompt_len
        break


  # Store the activations weights
  last_layer_activations = activations
  activations_total_list.append(last_layer_activations)
  # shape of activations_total_list:
  # (num_prompt, max_length, batch_size, prompt_len, 3200)


  activations_scores_qm.append(activations[-1][0][qm_idx] )
  activations_scores_a1.append(activations[-1][0][a1_idx] )
  activations_scores_a1vsqm.append(activations[a1_idx - prompt_len + 1][0][qm_idx])

  attention_required = attention_scores[-1][0, :, prompt_len, qm_idx]

save_data = {"activations_scores_qm": activations_scores_qm, "activations_scores_a1": activations_scores_a1, "activations_scores_a1vsqm": activations_scores_a1vsqm}

torch.save(save_data, data_path + f"research_data_mix/research_data_mix.pt") # store the result to google drive

100%|██████████| 500/500 [13:21<00:00,  1.60s/it]


### Classifier training_MLP
You need to get the artifact data in advance

In [ ]:
from google.colab import drive
import torch
import numpy as np
drive.mount('/content/drive', force_remount=True)
data_path = '/content/drive/My Drive/MSBD5002_project/' # modify this line for your drive

Mounted at /content/drive


In [ ]:
training_data = []
for i in range(18):
  training_data.append(torch.load(data_path + f"research_data_mix/research_data_mix_{500*(i+1)}.pt", map_location=torch.device('cpu'))) # shape (18, score, 500, score_len)

In [ ]:
# Reformatting the data
np_activations_scores_qm = np.stack([[y.cpu() for y in x["activations_scores_qm"]] for x in training_data]).reshape(-1, 4096)
np_activations_scores_a1 = np.stack([[y.cpu() for y in x["activations_scores_a1"]] for x in training_data]).reshape(-1, 4096)
np_activations_scores_a1vsqm = np.stack([[y.cpu() for y in x["activations_scores_a1vsqm"]] for x in training_data]).reshape(-1, 4096)
np_attention_required_ikey_ofirst_total_list = np.stack([[y.cpu() for y in x["attention_required_ikey_ofirst_total_list"]] for x in training_data]).reshape(-1, 32)
np_attention_required_ikey_olast_total_list = np.stack([[y.cpu() for y in x["attention_required_ikey_olast_total_list"]] for x in training_data]).reshape(-1, 32) # good result

In [ ]:
labels = []
for i in range(3):
  data_load = torch.load(data_path + f"attention_data_{1000*(i+1)}.pt") # it may takes around 1 minute
  labels.append(data_load['labels'])

In [ ]:
labels = np.array(labels).flatten()

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

labels_onehot = tf.keras.utils.to_categorical(labels, num_classes=2) # labels with one-hot-encoding



# Random 80/20 split
train_indices, test_indices = train_test_split(
    np.arange(len(labels)), test_size=0.2, random_state=21, shuffle=True
)

train_activation = np_attention_required_ikey_ofirst_total_list[train_indices]
train_labels = labels[train_indices]
train_labels_onehot = labels_onehot[train_indices]

test_activation = np_attention_required_ikey_ofirst_total_list[test_indices]
test_labels = labels[test_indices]
test_labels_onehot = labels_onehot[test_indices]

# Define model
input_dim = train_activation.shape[1]
hidden_dim = 256  # Smaller hidden layer for simpler classifier

model = Sequential([
    Dense(hidden_dim, input_dim=input_dim, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(hidden_dim//2, input_dim=input_dim, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(2, activation='sigmoid')  # Output dim=1 for binary classification, dim=2 for one-hot-encoding
])


# Training parameters
batch_size = 96
epochs = 200
lr = 0.0001
weight_decay = 0.05

# Compile model
optimizer = AdamW(learning_rate=lr, weight_decay=weight_decay)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)


# Training loop with evaluation every 10 epochs
for epoch in tqdm(range(epochs), desc="Training"):
    history = model.fit(
        train_activation, train_labels_onehot,
        batch_size=batch_size,
        epochs=1,
        verbose=0,
        shuffle=True
    )

    if (epoch + 1) % 10 == 0:
        train_loss = history.history['loss'][0]
        train_accuracy = history.history['accuracy'][0]

        # Evaluate on test set
        test_loss, test_accuracy = model.evaluate(
            test_activation, test_labels_onehot, verbose=0
        )
        # store the model for every 10 epoch
        model.save(data_path + "/MLP_model/" + f"model_epochs_{epoch}.keras")
        # Calculate AUROC
        test_outputs = model.predict(test_activation, verbose=0)[:, 1]
        auroc = roc_auc_score(test_labels, test_outputs)

        print(f'Epoch [{epoch+1}/{epochs}],'
              f'Train Loss: {train_loss:.4f},'
              f'Test Loss: {test_loss:.4f},'
              f'Test Accuracy: {test_accuracy:.4f},'
              f'Test AUROC: {auroc:.4f}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Training:   6%|▌         | 11/200 [00:05<01:37,  1.93it/s]

Epoch [10/200],Train Loss: 1.9714,Test Loss: 1.9386,Test Accuracy: 0.5700,Test AUROC: 0.5019


Training:  10%|█         | 21/200 [00:06<00:33,  5.31it/s]

Epoch [20/200],Train Loss: 1.3667,Test Loss: 1.3533,Test Accuracy: 0.5700,Test AUROC: 0.5285


Training:  16%|█▌        | 31/200 [00:08<00:29,  5.72it/s]

...

Epoch [70/200],Train Loss: 0.6864,Test Loss: 0.6957,Test Accuracy: 0.5700,Test AUROC: 0.5397


Training:  40%|████      | 81/200 [00:15<00:20,  5.67it/s]

Epoch [80/200],Train Loss: 0.6786,Test Loss: 0.6883,Test Accuracy: 0.5700,Test AUROC: 0.5338


Training:  44%|████▎     | 87/200 [00:16<00:21,  5.30it/s]


KeyboardInterrupt: 

### Classifier training_XGB
This part is independent to the MLP classifier

In [ ]:
from google.colab import drive
import torch
import numpy as np
drive.mount('/content/drive', force_remount=True)
data_path = '/content/drive/My Drive/MSBD5002_project/' # modify this line for your drive

Mounted at /content/drive


In [ ]:
training_data = []
for i in range(18):
  training_data.append(torch.load(data_path + f"research_data_mix/research_data_mix_{500*(i+1)}.pt", map_location=torch.device('cpu'))) # shape (18, score, 500, score_len)

In [ ]:
# Reformatting the data
np_activations_scores_qm = np.stack([[y.cpu() for y in x["activations_scores_qm"]] for x in training_data]).reshape(-1, 4096)
np_activations_scores_a1 = np.stack([[y.cpu() for y in x["activations_scores_a1"]] for x in training_data]).reshape(-1, 4096)
np_activations_scores_a1vsqm = np.stack([[y.cpu() for y in x["activations_scores_a1vsqm"]] for x in training_data]).reshape(-1, 4096)
np_attention_required_ikey_ofirst_total_list = np.stack([[y.cpu() for y in x["attention_required_ikey_ofirst_total_list"]] for x in training_data]).reshape(-1, 32)
np_attention_required_ikey_olast_total_list = np.stack([[y.cpu() for y in x["attention_required_ikey_olast_total_list"]] for x in training_data]).reshape(-1, 32) # good result

In [ ]:
labels = []
for i in range(3):
  data_load = torch.load(data_path + f"attention_data_{1000*(i+1)}.pt") # it may takes around 1 minute
  labels.append(data_load['labels'])

In [ ]:
labels = np.array(labels).flatten()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Random 80/20 split
train_indices, test_indices = train_test_split(
    np.arange(len(labels)), test_size=0.2, random_state=21, shuffle=True
)

train_activation = np_activations_scores_qm[train_indices]
train_labels = labels[train_indices]
test_activation = np_activations_scores_qm[test_indices]
test_labels = labels[test_indices]


xgb = XGBClassifier(n_estimators=800, learning_rate=0.011, max_depth=10 , eval_metric='auc', tree_method = "hist", device = "cuda")
xgb.fit(train_activation, train_labels)
test_outputs_xgb = xgb.predict_proba(test_activation)[:, 1]
auroc_xgb = roc_auc_score(test_labels, test_outputs_xgb)
print(f'XGBoost AUROC: {auroc_xgb:.4f} ')

XGBoost AUROC: 0.5000 i = 0
XGBoost AUROC: 0.5945 i = 1
XGBoost AUROC: 0.5946 i = 2
XGBoost AUROC: 0.5951 i = 3
XGBoost AUROC: 0.5962 i = 4
XGBoost AUROC: 0.5965 i = 5
XGBoost AUROC: 0.5969 i = 6
XGBoost AUROC: 0.5977 i = 7
XGBoost AUROC: 0.5974 i = 8


In [ ]:
# Save the model
xgb.save_model(data_path + f"xgbmodel_np_attention_required_ikey_olast_total_list_0.69.json")